原则是选择好而不同的学习器：


&emsp;&emsp;集成学习主要两个类别：

1. 序列方法
2. 并行方法

&emsp;&emsp;其中**提升方法是集成学习的一种**。提升方法的基本思路是：将弱可学习算法提升为强可学习算法。

## 提升方法

### Adaboost

- https://zhuanlan.zhihu.com/p/27126737

&emsp;&emsp;`Adaboost`算法解决的是分类问题，$y \in [-1, 1]$。

1. 在训练数据上训练得到模型，查看模型在整体数据和单个数据的分类效果。
2. 在整体数据上分类效果较好，则该模型在最后的模型中占较大比例，反之。
3. 在单个数据上分类效果较好，那么在训练下一个模型时，调小该单个数据的权值，反之。

&emsp;&emsp;在上面过程迭代$N$次之后，直到最后的分类结果达到预期目标。将所有的模型组合，得到强可学习模型。

&emsp;&emsp;`Adaboost`算法：

&emsp;&emsp;**输入**：训练数据集$T=\{(x_{1}, y_{1}), (x_{2}, y_{2}), \cdots, (x_{N}, y_{N})\}$, 其中$x_{i} \in X$, $y_{i} \in Y = \{-1, +1\}$；

&emsp;&emsp;**输出**：最终分类器$G_{(x)}$。


1. 初始化训练数据的权值分布：$D_{1}=(w_{11}, \cdots, w_{1i}), \ \ w_{1i}=\frac{1}{N}, \ i=1,2,\cdots, N$。
2. 对不同的基分类器: $m=1,2,\cdots, M$:

    (a) 使用具有权值分布$D_{m}$的训练数据集学习，得到基本分类器。$G_{m}(x):X\rightarrow \{-1, 1\}$。
    
    (b)计算$G_{m}(x)$在训练数据集上的分类误差率(所有分错了的样本对应的权值加起来)。
    $$
    e_{m} = \sum_{i=1}^{N} P(G_{m}(x_{i} \neq y_{i})) = \sum_{i=1}^{N} w_{mi} I(G_{m}(x_{i} \neq y_{i}))
    $$
    
    (c)计算$G_{m}(x)$在训练数据集上的分类误差。
    $$
    \alpha_{m} = \frac{1}{2} log \frac{1 - e_{m}}{e_{m}}
    $$
    (d)更新训练数据集的权值分布。
    $$
    D_{m+1}=(w_{m+1, 1}, \cdots, w_{m+1, i}, \cdots, w_{m+1, N})
    $$
    $$
    w_{m+1, i}=\frac{w_{mi}}{Z_{m}}exp(-\alpha_{m} y_{i} G_{m}(x_{i})), \ \ i=1,2,\cdots, N
    $$
    这里$Z_{m}$是规范化因子，它使$D_{m+1}$成为一个概率分布。$Z_{m}=\sum_{i=1}^{N}w_{mi}exp(-\alpha_{m}y_{i}G_{m}(x_{i}))$。

3. 构建基本分类器的先行组合

$$
f(x) = \sum_{m=1}^{M}\alpha_{m}G_{m}(x)
$$

&emsp;&emsp;得到最终分类器。

$$
G(x)=sign(f(x))\\
= sign(\sum_{m=1}^{M}\alpha_{m}G_{m}(x))
$$

### 提升树(Boosting Tree)

&emsp;&emsp;提升树中的每一个分类器都属于分类树，或者回归树(基分类器是决策树)。`Adaboost`的基分类器就是一个二分类。提升树模型的基分类器是决策树。

1. **提升树模型**：

$$
f_{M}(x)=\sum_{m=1}^{M} T\left(x, \Theta_{m}\right)
$$

&emsp;&emsp;比如需要去预测37摄氏度，第一颗树预测出来的结果是15摄氏度，第二颗树需要预测的是$37-15=22$摄氏度。但是第二颗树也不能预测准确，依次做下去。

2. **前向分步算法**：

$$
f_{m}(x)=f_{m-1}(x)+T\left(x, \Theta_{m}\right)
$$

&emsp;&emsp;在构建第$m$颗树的时候，前面$m-1$颗树固定不动。对决策树的参数$\Theta$的确定采用经验风险最小化来确定：

$$
\hat{\Theta}=\arg \min _{\Theta_{m}} \sum_{i=1}^{N} L\left(y_{i}, f_{m-1}(x)+T\left(x_{i} ; \Theta_{m}\right)\right)
$$

&emsp;&emsp;它整体的过程是在不停地减小偏差。

- 算法(**回归问题的提升树方法**)

&emsp;&emsp;输入：训练数据集$T=\{(x_{1}, y_{1}), (x_{2}, y_{2}), \cdots, (x_{N}, y_{N})\}$。

&emsp;&emsp;输出：提升树$f_{M}(x)$。

1. 初始化$f_{0}(x)=0$

2. 对$m=1,2,\cdots, M$

    (a) 计算残差：
    $$
    r_{mi}=y_{i}-f_{m-1}(x_{i}), \ \ i=1,2,\cdots, N
    $$
    (b) 拟合残差$r_{mi}$学习一个回归树，得到$T(x;\Theta)$
    
    (c) 更新$f_{m}(x)=f_{m-1}(x) + T(x;\Theta)$
3. 得到回归问题提升树

    $$
    f_{M}(x) = \sum_{m=1}^{M}T(x;\Theta)
    $$

    至于拟合残差的原因：对于任意样本点$y$和拟合值$f(x)$的损失$L(y, f(x))=[y-f(x)]^{2}$。在前项分布算法中：
    $$
    f_{m}(x)=[y-f_{m-1}(x)-T(x;\Theta_{m})]^{2}\\
    = [\gamma_{m-1}-T(x;\Theta_{m})]^{2}\\
    = L(\gamma_{m-1}, T(x;\Theta))
    $$

### GBDT

&emsp;&emsp;对于梯度提升树其学习流程与提升树类似只是不再使用残差作为新的训练数据而是使用损失函数的梯度作为新的训练数据的$y$值，具体的来说就是使用损失函数对$f(x)$求梯度然后带入$f_{m-1}(x)$计算：

- 算法(**梯度提升算法**)

&emsp;&emsp;输入：训练数据集$T=\{(x_{1}, y_{1}), (x_{2}, y_{2}), \cdots, (x_{N}, y_{N})\}$。

&emsp;&emsp;输出：回归树$\hat{f}(x)$。

1. 初始化: 寻找一个常数$c$来使得$f(x)$最小：

$$
f_{0}(x) = arg\min_{c} \sum_{i=1}^{N} L(y_{i}, c)
$$

2. 对于$m=1,2,\cdots, M$

    (a) 对$i=1,2,\cdots, N$计算
    $$
    r_{mi}=-\left[\frac{\partial L\left(y_{i}, f\left(x_{i}\right)\right)}{\partial f\left(x_{i}\right)}\right]_{f(x)=f_{m-1}(x)}
    $$
    (b) 对$r_{mi}$拟合一个回归树，得到第$m$颗树的叶节点区域$R_{mj}, j=1,2,\cdots, J$。
    (c) 对$j=1,2,\cdots, J$计算:
    $$
    c_{mj}=arg\min_{c} \sum_{x_{i} \in R_{mj}} L(y_{i}, f_{m-1}(x_{i}) + c)
    $$
    (d) 更新$f_{m}(x)=f_{m-1}(x) + \sum_{j=1}^{J}c_{mj}I(x \in R_{mj})$。

3. 得到回归树

$$
\hat{f}(x) = f_{M}(x) = \sum_{m=1}^{M}\sum_{j=1}^{J} c_{mj}I(x \in R_{mj})
$$

&emsp;&emsp;在使用平方误差损失函数和指数损失函数时，提升树的残差求解比较简单，但是在使用一般的损失误差函数时，残差求解起来不是那么容易，所以就是用损失函数的负梯度在当前模型的值作为回归问题中残差的近似值。 

### XGB

### XGBoost

梯度提升树。

### LightBoost

## Bagging方法

&emsp;&emsp;直接训练多个并行的分类器。对于最后的结果，如果是分类问题就做一个投票，如果是一个回归问题就做一个加权平均。它可以显著减小方差。

&emsp;&emsp;整体模型的方差可以用公式表示：

$$
\sigma^{2} = \rho \sigma^{2} + (1-\rho)\frac{\sigma^{2}}{n}
$$

&emsp;&emsp;因为每次是从数据集合中随机抽样，那么假设数据之间的相关性$\rho=0$, 那么最后的方差则可以表示为$\sigma^{2}= + \frac{\sigma^{2}}{n}$。

&emsp;&emsp;`Boosting`方法假设其强相关，那么$\sigma^{2}= \sigma^{2}$，并没有一个方差的显著降低。